## MNIST classification

In [ ]:
from CNN.mnist_model import ConvNet
from CNN.utils import get_weights_biases
import torch
import time
from torchvision import datasets, transforms
import quartz
from quartz import layers
import numpy as np

In [ ]:
n_classes = 10
model = ConvNet(n_classes)
# model.load_state_dict(torch.load("CNN/mnist-convnet.pth", map_location=torch.device('cpu')))
model.load_state_dict(torch.load("CNN/mnist-convnet-percentile.pth", map_location=torch.device('cpu')))
model.eval()

In [ ]:
weights, biases = get_weights_biases(model)

In [ ]:
([weight.max() for weight in weights])

In [ ]:
([weight.min() for weight in weights])

In [ ]:
weights[3] /= 3
biases[3] /= 3

## build model

In [ ]:
t_max = 2**6
input_dims = (1,28,28)
pool_kernel_size = [2,2]
batch_size = 2500

loihi_model = quartz.Network(t_max, [
    layers.InputLayer(dims=input_dims),
    layers.Conv2D(weights=weights[0], biases=biases[0]),
    layers.MaxPool2D(kernel_size=pool_kernel_size),
    layers.Conv2D(weights=weights[1], biases=biases[1]),
    layers.MaxPool2D(kernel_size=pool_kernel_size),
    layers.Conv2D(weights=weights[2], biases=biases[2]),
    layers.Dense(weights=weights[3], biases=biases[3], rectifying=False),
])

In [ ]:
loihi_model

In [ ]:
#loihi_model.check_block_delays(t_max, 2**3)
#loihi_model.print_core_layout(redo=True)

## load data

In [ ]:
transform=transforms.Compose([transforms.ToTensor(),])
test_set = datasets.MNIST('./CNN/data', train=False, download=False, transform=transform)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=False)

In [ ]:
test_loader_iter = iter(test_loader)

In [ ]:
print(time.strftime("Started on %a, %d %b %Y %H:%M:%S", time.gmtime()))
start_time = time.time()

errors = []
for inputs, target in test_loader_iter:
    loihi_output = loihi_model(inputs.detach().numpy(), partition='loihi_2h', logging=False, profiling=False)
    firsts = np.zeros((n_classes,batch_size))
    for i, (key, values) in enumerate(sorted(loihi_model.data[0].items())[1:n_classes+1]):
        for iteration in range(batch_size):
            firsts[i, iteration] = values[(values>(iteration * loihi_model.steps_per_image)) & (values<((iteration+1)*loihi_model.steps_per_image))][0]
    loihi_results = np.argmin(firsts, axis=0)
    positives = sum(loihi_results == target.numpy())
    negatives = loihi_results != target.numpy()
    error = 100*(1-positives/len(target))
    errors.append(error)
    print("Correctly detected {} out of {}: {}% error".format(positives, len(target), error))
    break

print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
#loihi_output

In [ ]:
firsts

In [ ]:
#loihi_output.reshape(loihi_model.layers[-2].output_dims, batch_size).T

In [ ]:
#loihi_model.data

In [ ]:
#firsts


In [ ]:
#sorted(loihi_model.data[1].items())[:batch_size]

In [ ]:
#inputs

In [ ]:
#loihi_output.rawPowerTimeStamps

In [ ]:
np.sum(errors)/len(errors)

In [ ]:
with open("results.txt", "a") as myfile:
    myfile.write("{}\n".format(errors))

In [ ]:
#firsts.min()

In [ ]:
#np.max(loihi_output)

In [ ]:
%debug